In [1]:
# Football API
from espn_api.football import League

#import necessary packages
import pandas as pd
import numpy as np
import requests

In [2]:
# Enter the desired league id
league_id = 214643

# Creating DataFrame
df1 = pd.DataFrame()

# Creating a for loop to populate df1 with standings for each year of the league's history (2010-2019)
for i in range(2010, 2020, 1):

    # Writing the url to input custom league id and grab data from each year
    url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
          str(league_id) + "?seasonId=" + str(i)

    # Using requests package to locate the proper data (we want team data for league standings)
    r = requests.get(url, params={"view": "mTeam"})

    # Converting data into a json file from "response" output
    d = r.json()[0]

    # Populating df2 with the desired data from the json file
    df2 = [[team['primaryOwner'], team['location'] + ' ' + team['nickname'], team['record']['overall']['wins'],
            team['record']['overall']['losses'], team['record']['overall']['ties'], 
            team['record']['overall']['percentage'], team['record']['overall']['pointsFor'], 
            team['record']['overall']['pointsAgainst'], team['transactionCounter']['acquisitions'], 
            team['transactionCounter']['trades'], team['transactionCounter']['drops']] 
           
            for team in d['teams']]
    
    # Creating a 2nd DataFrame and naming each column to make the data much easier to read and clean
    df2 = pd.DataFrame(df2, columns=['OID', 'Team Name', 'W', 'L', 'T', 'PCT', 'PF', 'PA', 'Moves', 'Trades', 'Drops'])
    
    # Inserting league year into each row
    df2.insert(2, "Year", d['seasonId'])
    
    # Using and appending df1 (separate DataFrame) to populate every year in the for loop
    # A separate DataFrame is used to stop the for loop from replacing the old data with next year's data
    df1 = df1.append(df2)

# Calculating Points For & Against per Game and Avg. Point Differential
# Inserting these columns into the DataFrame
df1.insert(9, "PF/G", df1['PF'] / (df1['W'] + df1['L'] + df1['T']))
df1.insert(10, "PA/G", df1['PA'] / (df1['W'] + df1['L'] + df1['T']))
df1.insert(11, "DIFF", df1['PF/G'] - df1['PA/G'])

In [3]:
# Removing {} to clean unique keys for each owner
df1['OID'].str[1:-1]

# Rounding numbers to the desired decimal places
df1['PCT'] = df1['PCT'].round(3)
df1[['PF','PA']] = df1[['PF','PA']].round()
df1[['PF/G','PA/G', 'DIFF']] = df1[['PF/G','PA/G', 'DIFF']].round(2)

# Converting columns from floats to integers to remove decimal places completely
df1[['PF','PA']] = df1[['PF','PA']].astype(int)

df1.head()

,OID,Team Name,Year,W,L,T,PCT,PF,PA,PF/G,PA/G,DIFF,Moves,Trades,Drops
0,{A5C5314E-88CA-46AD-9193-5C266CA0907B},Oyster Bay Jets,2010,10,3,0,0.769,1258,1064,96.77,81.85,14.92,22,5,24
1,{F3826AB6-2D8C-4A3D-87EB-E35C18F9B4A6},Italian Umenyioras,2010,5,8,0,0.385,1151,1229,88.54,94.54,-6.00,24,2,27
2,{AC7CD8FD-DF42-47AB-8386-C7538B73377A},New York Ballers,2010,6,6,1,0.500,1108,1191,85.23,91.62,-6.38,34,6,32
3,{48977A0D-2297-4D66-8E14-6583D6878270},MEAN MACHINE,2010,7,5,1,0.577,1140,1117,87.69,85.92,1.77,2,0,2
4,{8C0C3E4F-06D8-4219-AD7F-16755C7CC01A},JAMICAN JAWANS,2010,5,8,0,0.385,999,1034,76.85,79.54,-2.69,0,0,0


In [5]:
df1.to_excel("AntoniosLeagueStandings.xlsx")